In [25]:
import os
from PIL import Image
import numpy as np

In [26]:
x,y = load_data('F:/Project/Processed_Data/Mouth_Images_Random/Addresses_Random.txt',3,1)

NameError: name 'load_data' is not defined

In [27]:
def load_data(path,num_word,times):
    labels = []
    inputs = []
    with open(path,'r') as f:
        lines = f.readlines()
        count = 0
        label_count = 0
        target_lines = lines[times*25*num_word:times*25*num_word+num_word*25]
        for i in target_lines:           
            #print(i)
            if(count>=(num_word*25)):
                break
            #temp = Image.open(i.strip('\n')).convert('L')
            temp = Image.open(i.strip('\n'))
            #image_t = np.array(temp)
            image_t = np.array(temp)
            inputs.append(image_t)
            temp.close()
            
            if((label_count == 0)|(label_count == 25)):
                label_count = 0
                all_string = i.split('/')
                #print(all_string[-1])
                #print(all_string)
                labels.append((all_string[-1]).split('_')[-2])
                #print(labels[-1])
            label_count += 1
            count += 1
            #print(all_string[-1])
    #print(labels)
    x = np.asarray(inputs, dtype = np.float32)
    #x = x/255.0
    return x,labels

In [19]:
x,y = load_data('F:/Project/Process_Small_Python/bin_please/train.txt',2,1)
print(x.shape)


(50, 80, 100, 3)


In [28]:
def one_hot(label):
    
    word_map = ['bin','lay','place','set',
               'blue','red','green','white',
               'at','in','with','by',
               'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','x','y','z',
               'zero','one','two','three','four','five','six','seven','eight','nine',
               'again','now','soon','please']
    word_to_int = dict((c,i) for i,c in enumerate(word_map))
    #print(word_to_int)
    int_to_word = dict((i,c) for i,c in enumerate(word_map))
    #print(target[1])
    #print(word_to_int[target[1]])
    interger_encoded = [word_to_int[word] for word in label]
    
    onehot_encoded = list()
    for value in interger_encoded:
        letter = [0 for _ in range(len(word_map))]
        letter[value] = 1
        onehot_encoded.append(letter)
    result = np.asarray(onehot_encoded,dtype = np.int32)
    return result

In [29]:
def reshape(x,y):
    return(x.reshape(-1,25,80,100,3),y.reshape(-1,51))

In [30]:
def data_generator(path,steps_per_epoch,epochs):
    #for ii in range(2):
    for ii in range(epochs):
        for i in range(int(114000/steps_per_epoch)):
        
            x,y = load_data(path,steps_per_epoch,i)
            output = one_hot(y)
            inputs,label = reshape(x,output)
            yield(inputs,label)

In [36]:
from keras.layers import Dense, LSTM, TimeDistributed,Convolution2D,MaxPooling2D,Activation,Flatten,BatchNormalization,Dropout
from keras.models import Sequential
from keras import optimizers
from keras.layers.wrappers import TimeDistributed
from keras.layers.core import Reshape
from keras import optimizers
import tensorflow as tf
import matplotlib.pyplot as plt
import keras.backend as K
from sklearn.metrics import classification_report, confusion_matrix

In [32]:
model = Sequential()
model.add(TimeDistributed(Convolution2D(96, (3, 3),padding='same'),
                          input_shape=(25,80,100,3)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(3, 3))))

model.add(TimeDistributed(Convolution2D(256, (3, 3),padding='same')))
model.add(BatchNormalization())
#model.add(Activation('relu'))
model.add(TimeDistributed(MaxPooling2D(pool_size=(3, 3))))
#model.add(Dropout(0.2))

model.add(TimeDistributed(Convolution2D(512, (3, 3),padding='same')))
#model.add(BatchNormalization())
model.add(TimeDistributed(Convolution2D(512, (3, 3),padding='same')))
#model.add(BatchNormalization())
model.add(TimeDistributed(Convolution2D(512, (3, 3),padding='same')))
#model.add(BatchNormalization())
model.add(TimeDistributed(MaxPooling2D(pool_size=(3, 3))))

model.add(TimeDistributed(Flatten()))
model.add(LSTM(512, return_sequences=True, name="lstm_layer"))
model.add(TimeDistributed(Dense(256,activation='softmax'), name="time_distr_dense_one"))

model.add(LSTM(512, return_sequences=False, stateful=False))
#model.add(TimeDistributed(Dense(256,activation='softmax'), name="time_distr_dense_one2"))
model.add(Dense(2048))
model.add(((Dense(51,activation='softmax'))))
Adam = optimizers.Adamax(lr=0.05, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer='Adam',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 25, 80, 100, 96)   2688      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 25, 26, 33, 96)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 25, 26, 33, 256)   221440    
_________________________________________________________________
batch_normalization_1 (Batch (None, 25, 26, 33, 256)   1024      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 25, 8, 11, 256)    0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 25, 8, 11, 512)    1180160   
_________________________________________________________________
time_distributed_6 (TimeDist (None, 25, 8, 11, 512)    2359808   
__________

In [8]:
#model = Sequential()
model.load_weights('first_two38.h5')
#K.set_value(Adam.lr, (1-((1e-6)*14)) * K.get_value(Adam.lr))

In [33]:
def data_generator_test(path,num_word):
    for ii in range(1):
        for i in range(6000):
            x,y = load_data(path,num_word,i)
            output = one_hot(y)
            #print("111")0
            inputs,label = reshape(x,output)
            yield (inputs,label)

In [9]:
num_word = 20
epochs = 40
for epoch in range(epochs):
    history = model.fit_generator(data_generator('F:/Project/Process_Small_Python/all_words_random/train.txt',num_word,epochs), verbose=1, workers=2,
                              steps_per_epoch=int(114000/num_word), epochs = 1)
    time = epoch
    model.save_weights('first_two'+str(time)+'.h5')
    score = model.evaluate_generator(data_generator_test('F:/Project/Process_Small_Python/all_words_random/test.txt'
                                                    ,20),steps=600)
    print(score)
    #K.set_value(Adam.lr, (1-(1e-4)) * K.get_value(Adam.lr))

Epoch 1/1
5700/5700 [==============================] - 7820s 1s/step - loss: 3.1398 - acc: 0.1281
[2.9678128027915953, 0.13250000272567072]
Epoch 1/1
5700/5700 [==============================] - 8574s 2s/step - loss: 3.0709 - acc: 0.1424
[2.984708185195923, 0.13191666934639215]
Epoch 1/1
5700/5700 [==============================] - 8199s 1s/step - loss: 3.0590 - acc: 0.1456
[2.976596350669861, 0.13116666942213973]
Epoch 1/1
5700/5700 [==============================] - 7809s 1s/step - loss: 3.0576 - acc: 0.1467
[2.964789400100708, 0.12683333606148758]
Epoch 1/1
5700/5700 [==============================] - 7624s 1s/step - loss: 3.0555 - acc: 0.1476
[2.9602037886778514, 0.12775000264247258]
Epoch 1/1
5700/5700 [==============================] - 7609s 1s/step - loss: 3.0525 - acc: 0.1479
[2.941417522033056, 0.13075000268096726]
Epoch 1/1
5700/5700 [==============================] - 7565s 1s/step - loss: 3.0406 - acc: 0.1529
[2.9518502163887024, 0.13325000268096726]
Epoch 1/1
5700/5700 [===

In [50]:
score = model.predict_generator(data_generator_test('F:/Project/Process_Small_Python/all_words_random/test.txt'
                                                    ,20),steps=600)
print(score.shape)

(12000, 51)


In [9]:
def plot():
    loss = history.history['loss']
    val_loss = history.history['acc']
    plt.plot(loss)
    plt.plot(val_loss)
    plt.legend(['loss', 'acc'])
    plt.show()

In [12]:
print("11111")
#output = one_hot(y)
#x_test,y_test = reshape(x,output,25)
score = model.evaluate_generator(data_generator_test('F:/Project/Process_Small_Python/all_words_test/test.txt'
                                                    ,2),steps=3000)

print(score)

11111
[2.7000083893934885, 0.21283333333333335]


In [20]:
def load_y(path):
    labels = []
    #inputs = []
    with open(path,'r') as f:
        lines = f.readlines()
        print(len(lines))
        count = 0
        label_count = 0
        #target_lines = lines
        for i in lines:           
            #print(i)
            #if(count>=(num_word*25)):
                #break
            #temp = Image.open(i.strip('\n')).convert('L')
            #temp = Image.open(i.strip('\n'))
            #image_t = np.array(temp)
            #image_t = np.array(temp)
            #inputs.append(image_t)
            #temp.close()
            
            if((label_count == 0)|(label_count == 25)):
                label_count = 0
                all_string = i.split('/')
                #print(all_string[-1])
                #print(all_string)
                labels.append((all_string[-1]).split('_')[-2])
                #print(labels[-1])
            label_count += 1
            count += 1
            #print(all_string[-1])
    #print(labels)
    #x = np.asarray(inputs, dtype = np.float32)
    #x = x/255.0
    return labels

In [24]:
only_y = load_y('F:/Project/Process_Small_Python/all_words_random/test.txt')
print(len(y))
hot_y = one_hot(only_y)

300000
12000


In [23]:
def one_hot(label):
    
    word_map = ['bin','lay','place','set',
               'blue','red','green','white',
               'at','in','with','by',
               'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','x','y','z',
               'zero','one','two','three','four','five','six','seven','eight','nine',
               'again','now','soon','please']
    word_to_int = dict((c,i) for i,c in enumerate(word_map))
    #print(word_to_int)
    int_to_word = dict((i,c) for i,c in enumerate(word_map))
    #print(target[1])
    #print(word_to_int[target[1]])
    interger_encoded = [word_to_int[word] for word in label]
    
    onehot_encoded = list()
    for value in interger_encoded:
        letter = [0 for _ in range(len(word_map))]
        letter[value] = 1
        onehot_encoded.append(letter)
    result = np.asarray(onehot_encoded,dtype = np.int32)
    return result

In [48]:
print(hot_y[1])
print(score[1])
matrix = confusion_matrix(hot_y.argmax(axis=1), score.argmax(axis=1))
print(matrix[0])

[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0.01955224 0.01958951 0.0196271  0.0195902  0.01956972 0.0196392
 0.01957088 0.01965452 0.019557   0.01966373 0.0196252  0.01961722
 0.0195812  0.01959906 0.01962689 0.0196409  0.01964178 0.01955035
 0.01962658 0.01956737 0.01961608 0.01960778 0.01962405 0.01956657
 0.01955927 0.01960347 0.01966549 0.01958907 0.01962623 0.01958452
 0.0196307  0.01961634 0.01963692 0.01963503 0.0195704  0.01963333
 0.01963364 0.01961233 0.01959748 0.01957119 0.01962866 0.01958621
 0.0196631  0.01957931 0.01958702 0.01963642 0.01955977 0.01964775
 0.01964326 0.01963185 0.01956612]
[  0   0   0   0   0   0   0   0   0  79   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0  34   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0 399   0   0   0   0   0   0   0   0]


In [39]:
word_map = ['bin','lay','place','set',
               'blue','red','green','white',
               'at','in','with','by',
               'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','x','y','z',
               'zero','one','two','three','four','five','six','seven','eight','nine',
               'again','now','soon','please']

In [49]:
print(classification_report(hot_y.argmax(axis=1),score.argmax(axis=1), target_names=word_map))

             precision    recall  f1-score   support

        bin       0.00      0.00      0.00       512
        lay       0.00      0.00      0.00       512
      place       0.00      0.00      0.00       483
        set       0.00      0.00      0.00       493
       blue       0.00      0.00      0.00       500
        red       0.00      0.00      0.00       500
      green       0.00      0.00      0.00       500
      white       0.00      0.00      0.00       500
         at       0.00      0.00      0.00       501
         in       0.00      0.00      0.00       504
       with       0.00      0.00      0.00       499
         by       0.00      0.00      0.00       496
          a       0.00      0.00      0.00        80
          b       0.00      0.00      0.00        80
          c       0.00      0.00      0.00        80
          d       0.00      0.00      0.00        80
          e       0.00      0.00      0.00        80
          f       0.00      0.00      0.00   

F:\Softwares\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
